In [272]:
import sys 
import os
import time

rel_do_mpc_path = os.path.join('..','..')
sys.path.append(rel_do_mpc_path)

import do_mpc
from do_mpc.data import save_results, load_results
from matplotlib.animation import FuncAnimation, PillowWriter

import time
import numpy as np

from casadi import *
from casadi.tools import *

-----------------------
### <center>PARAMETRES UTILISATEURS</center>
-----------------------

In [273]:
auteur = 'step'   # Pour ne pas ecraser entre nous

# Environment
scenario = 1

# Metrique: J1 ou J2
use_objective = 'J1'

# Qualité du solveur
n_horizon = 30     # 10, 20 ou 50
t_step_ech = 0.05  # 0.05

In [274]:
results = {}  # Dictionary to store results
computation_times = {}  # Dictionary to store computation times

In [275]:
# ATTENTION : si un sénario est changé, il faut supprimé toute la data qui y est liée
use_obstacle = True
use_rterm2 = False
reference = 'cste'   # 'cste', 'bi', 'continuous', 'var' (t_switch: temps auquel on passe à la 2e consigne)

# CI : CInulles, CI1, CIcontournement, CIcontournement_replie
if scenario==1: # VEROUILLE <=====================================
    CI = 'CIcontournement'
    n_step_sim = 150
    consigne1 = np.pi/2
    gamma = np.pi

elif scenario==2: 
    CI = 'CIcontournement_replie'
    n_step_sim = 200
    consigne1 = np.pi
    gamma = np.pi

elif scenario==3: # VEROUILLE <=====================================
    CI = 'CI1'
    consigne1 = np.pi/2
    reference = 'bi'
    n_step_sim = 200
    t_switch = 2.5
    consigne2 = np.pi
    gamma = np.pi

elif scenario==4: # VEROUILLE <=====================================
    CI = 'CI1'
    use_rterm2 = True
    consigne1 = np.pi/2
    reference = 'bi'
    n_step_sim = 200
    t_switch = 2.5
    consigne2 = np.pi
    gamma = np.pi

elif scenario==5:  # VEROUILLE <=====================================
    CI = 'CI1'
    consigne1 = np.pi/2
    reference = 'continuous'
    v_rot = 2*np.pi/50   # rad/s
    n_step_sim = 1000
    gamma = np.pi

elif scenario==6:  # VEROUILLE <=====================================
    CI = 'CI1'
    consigne1 = np.pi/2
    reference = 'var'
    a_rot = 2*np.pi/50 /8   # rad/s²
    n_step_sim = 1000
    gamma = np.pi

else:
    raise ValueError('Scenario non reconnu')

In [276]:
# Define obstacles
obstacle = {'x': 0., 'y': 0, 'r': 0.4}

# Defini cercle bleu
cercle_bleu = {'x': obstacle['x'], 'y': obstacle['y'], 'r': 0.6}

-----------------------
### <center>MODELISATION</center>
-----------------------

In [277]:
# Creation du modele
model_type = "continuous"
model = do_mpc.model.Model(model_type)

In [278]:
# Pramètre physiques
l1 = 0.4; l2 = 0.4; l3 = 0.3
d0_min=0.4; d1_min=0.3; d2_min=0.1; d3_min=0.15

In [279]:
# System state variables
x1 = model.set_variable(var_type='_x', var_name='x1', shape=(1,1))
x2 = model.set_variable(var_type='_x', var_name='x2', shape=(1,1))
x3 = model.set_variable(var_type='_x', var_name='x3', shape=(1,1))
x4 = model.set_variable(var_type='_x', var_name='x4', shape=(1,1))
x5 = model.set_variable(var_type='_x', var_name='x5', shape=(1,1))

v1 = model.set_variable(var_type='_x', var_name='v1', shape=(1,1))
v2 = model.set_variable(var_type='_x', var_name='v2', shape=(1,1))
v3 = model.set_variable(var_type='_x', var_name='v3', shape=(1,1))
v4 = model.set_variable(var_type='_x', var_name='v4', shape=(1,1))

a1 = model.set_variable(var_type='_x', var_name='a1', shape=(1,1))
a2 = model.set_variable(var_type='_x', var_name='a2', shape=(1,1))
a3 = model.set_variable(var_type='_x', var_name='a3', shape=(1,1))
a4 = model.set_variable(var_type='_x', var_name='a4', shape=(1,1))

# System input variables
u1 = model.set_variable(var_type='_u', var_name='u1', shape=(1,1))
u2 = model.set_variable(var_type='_u', var_name='u2', shape=(1,1))
u3 = model.set_variable(var_type='_u', var_name='u3', shape=(1,1))
u4 = model.set_variable(var_type='_u', var_name='u4', shape=(1,1))

In [280]:
# Modèle dynamique du robot
model.set_rhs('x1', v1*np.cos(x3))
model.set_rhs('x2', v1*np.sin(x3))
model.set_rhs('x3', v2)
model.set_rhs('x4', v3)
model.set_rhs('x5', v4)

model.set_rhs('v1', a1)
model.set_rhs('v2', a2)
model.set_rhs('v3', a3)
model.set_rhs('v4', a4)

model.set_rhs('a1', u1)
model.set_rhs('a2', u2)
model.set_rhs('a3', u3)
model.set_rhs('a4', u4)

In [281]:
# Definition de la position du robot
p_x = model.set_expression('p_x', x1 + l1*np.cos(x3) + l2*np.cos(x3 + x4) + l3*np.cos(x3 + x4 + x5))
p_y = model.set_expression('p_y', x2 + l1*np.sin(x3) + l2*np.sin(x3 + x4) + l3*np.sin(x3 + x4 + x5))
theta = model.set_expression('theta', x3 + x4 + x5)

In [282]:
# Desfiniton des "time-varying parameters"
theta_set = model.set_variable('_tvp', 'theta_set')
theta_cmd = model.set_expression('theta_cmd', theta_set - np.pi)
p_x_set = model.set_expression('p_x_set', cercle_bleu['r']*np.cos(theta_set))
p_y_set = model.set_expression('p_y_set', cercle_bleu['r']*np.sin(theta_set))

In [283]:
# Definition des contraintes de non-collision
node0_x = model.x['x1'] 
node0_y = model.x['x2']
node1_x = node0_x+l1*np.cos(model.x['x3'])
node1_y = node0_y+l1*np.sin(model.x['x3'])
node2_x = node1_x+l2*np.cos(model.x['x3']+model.x['x4'])
node2_y = node1_y+l2*np.sin(model.x['x3']+model.x['x4'])

d0 = model.set_expression('d0', np.sqrt( (node0_x-obstacle['x'])**2 + (node0_y-obstacle['y'])**2 ) - (obstacle['r']+0.4)) 
d1 = model.set_expression('d1', np.sqrt( (node1_x-obstacle['x'])**2 + (node1_y-obstacle['y'])**2 ) - (obstacle['r']+0.3))
d2 = model.set_expression('d2', np.sqrt( (node2_x-obstacle['x'])**2 + (node2_y-obstacle['y'])**2 ) - (obstacle['r']+0.1))
d3 = model.set_expression('d3', np.sqrt( (p_x-obstacle['x'])**2     + (p_y-obstacle['y'])**2     ) - (obstacle['r']+0.15))

obstacle_distance = [d0, d1, d2, d3]
obstacle_distance = model.set_expression('obstacle_distance', vertcat(*obstacle_distance))

In [284]:
# Mise en place du modèle
model.setup()

-----------------------
### <center>CONFIGURATION DE L'OPTIMISEUR</center>
-----------------------

In [285]:
# Creation de l'optimiseur
mpc = do_mpc.controller.MPC(model)

In [286]:
# Optimizer paramters
setup_mpc = {
    'n_horizon': n_horizon,
    't_step': t_step_ech,
    'n_robust': 0,
    'store_full_solution': True,
}
mpc.set_param(**setup_mpc)

In [287]:
# Critères d'optimisation
err_1 = model.aux['p_x'] - model.aux['p_x_set']
err_2 = model.aux['p_y'] - model.aux['p_y_set']
err_3 = model.aux['theta'] - (model.tvp['theta_set'] - np.pi)   # Permet de pointer vers "l'interieur"

J1 = err_1**2 + err_2**2 + err_3**2
J2 = err_1**2 + err_2**2 + err_3**2 + v3**2 + v4**2
if use_objective == 'J1':
    J = J1
elif use_objective == 'J2':
    J = J2
else:
    raise ValueError('Unknown objective function')

# Mise en place des critères d'optimisation
mterm = J # Terminal cost
lterm = J # Stage cost
mpc.set_objective(mterm=mterm, lterm=lterm)

# Penalisation des variation de l'entree - IMPORTANT
if use_rterm2:
    mpc.set_rterm(
        u1=10,
        u2=5,
        u3=5,
        u4=0.1
    )
else:
    mpc.set_rterm(
        u1=0.1,
        u2=0.1,
        u3=0.1,
        u4=0.1
    )

In [288]:
# Definition de la variation de la position de l'effecteur
tvp_template_optim = mpc.get_tvp_template()

def pos_effecteur(t_ind):
    if reference == 'cste':
        tvp_template_optim['_tvp',:, 'theta_set'] = consigne1
    elif reference == 'bi':
        ind_switch = t_switch // mpc.settings.t_step
        ind = t_ind // mpc.settings.t_step
        if ind <= ind_switch:
            tvp_template_optim['_tvp',:, 'theta_set'] = consigne1
        else:
            tvp_template_optim['_tvp',:, 'theta_set'] = consigne2
        return tvp_template_optim
    elif reference == 'continuous':
        agl = consigne1 + t_ind*v_rot
        tvp_template_optim['_tvp',:, 'theta_set'] = agl
    elif reference == 'var':
        v_rot = a_rot*t_ind
        agl = consigne1 + t_ind*v_rot
        tvp_template_optim['_tvp',:, 'theta_set'] = agl
    else:
        raise ValueError('Commande effecteur non reconnue')
    
    return tvp_template_optim

mpc.set_tvp_fun(pos_effecteur)

In [289]:
# Mise en place des contraintes
# Limites sur les positions
mpc.bounds['lower','_x','x1'] = -10
mpc.bounds['upper','_x','x1'] = 10
mpc.bounds['lower','_x','x2'] = -10
mpc.bounds['upper','_x','x2'] = 10
# mpc.bounds['lower','_x','x3'] = -gamma
# mpc.bounds['upper','_x','x3'] = gamma
mpc.bounds['lower','_x','x4'] = -gamma
mpc.bounds['upper','_x','x4'] = gamma
mpc.bounds['lower','_x','x5'] = -gamma
mpc.bounds['upper','_x','x5'] = gamma

# Limites sur la vitesse
mpc.bounds['lower','_x','v1'] = -1.5
mpc.bounds['upper','_x','v1'] = 1.5
mpc.bounds['lower','_x','v2'] = -1.5
mpc.bounds['upper','_x','v2'] = 1.5
mpc.bounds['lower','_x','v3'] = -1.5
mpc.bounds['upper','_x','v3'] = 1.5
mpc.bounds['lower','_x','v4'] = -1.5
mpc.bounds['upper','_x','v4'] = 1.5

# Limite sur l'accélération (et donc les couples)
mpc.bounds['lower','_u','u1'] = -2
mpc.bounds['upper','_u','u1'] = 2
mpc.bounds['lower','_u','u2'] = -2
mpc.bounds['upper','_u','u2'] = 2
mpc.bounds['lower','_u','u3'] = -2
mpc.bounds['upper','_u','u3'] = 2
mpc.bounds['lower','_u','u4'] = -2
mpc.bounds['upper','_u','u4'] = 2

In [290]:
# Mise à l'echelle (améliore stabilité, convergence et comparabilité des paramètres de l'optimisation)
# Scaling des positions
mpc.scaling['_x', 'x1'] = 10  
mpc.scaling['_x', 'x2'] = 10  
# mpc.scaling['_x', 'x3'] = gamma
mpc.scaling['_x', 'x4'] = gamma  
mpc.scaling['_x', 'x5'] = gamma
# Scaling des vitesses
mpc.scaling['_x', 'v1'] = 1.5  
mpc.scaling['_x', 'v2'] = 1.5  
mpc.scaling['_x', 'v3'] = 1.5  
mpc.scaling['_x', 'v4'] = 1.5  
# Scaling des accélérations (ou couples)
mpc.scaling['_u', 'u1'] = 2
mpc.scaling['_u', 'u2'] = 2
mpc.scaling['_u', 'u3'] = 2
mpc.scaling['_u', 'u4'] = 2

In [291]:
# Eviter les obstacles
if use_obstacle:
    obstacles_mps = mpc.set_nl_cons('obstacles', -model.aux['obstacle_distance'], 0)

In [292]:
# Mise en place du parametrage de l'optimiseur
mpc.setup()

-----------------------
### <center>CONFIGUARATION DE L'ESTIMATEUR</center>
-----------------------
On fait l'hypothèse d'un retour direct

In [293]:
# Creation de l'estimateur
estimator = do_mpc.estimator.StateFeedback(model)

-----------------------
### <center>CONFIGURATION DU SIMULATEUR</center>
-----------------------

In [294]:
# Creation du simulateur
simulator = do_mpc.simulator.Simulator(model)

In [295]:
# Parametrage du simulateur
params_simulator = {
    'integration_tool': 'idas',
    'abstol': 1e-8,
    'reltol': 1e-8,
    't_step': t_step_ech
}
simulator.set_param(**params_simulator)

In [296]:
# Definition de la variation de la position de l'effecteur
tvp_template_sim = simulator.get_tvp_template()
def tvp_fun(t_ind):
    return tvp_template_sim
simulator.set_tvp_fun(tvp_fun)

In [297]:
# Initialisation du simulateur
simulator.setup()

-----------------------
### <center>PREPARATION DE L'AFFICHAGE GRAPHIQUE</center>
-----------------------

In [298]:
# Preparation du module graphique
mpc_graphics = do_mpc.graphics.Graphics(mpc.data)

In [299]:
# Preparation de matplotlib
import matplotlib.pyplot as plt
plt.ion()
from matplotlib import rcParams
rcParams['text.usetex'] = False
rcParams['axes.grid'] = True
rcParams['lines.linewidth'] = 2.0
rcParams['axes.labelsize'] = 'xx-large'
rcParams['xtick.labelsize'] = 'xx-large'
rcParams['ytick.labelsize'] = 'xx-large'
rcParams['font.size'] = 8

from matplotlib.patches import Circle

In [300]:
def robot(x):
    x = x.flatten()

    chassis_x = np.array([
        x[0],
        x[0] + l1*np.cos(x[2])
    ])
    chassis_y = np.array([
        x[1],
        x[1] + l1*np.sin(x[2])
    ])

    barre_intermediaire_x = np.array([
        chassis_x[1],
        chassis_x[1] + l2*np.cos(x[2]+x[3])
    ])
    barre_intermediaire_y = np.array([
        chassis_y[1],
        chassis_y[1] + l2*np.sin(x[2]+x[3])
    ])

    effecteur_x = np.array([
        barre_intermediaire_x[1],
        barre_intermediaire_x[1] + l3*np.cos(x[2]+x[3]+x[4])
    ])
    effecteur_y = np.array([
        barre_intermediaire_y[1],
        barre_intermediaire_y[1] + l3*np.sin(x[2]+x[3]+x[4])
    ])  

    chassis = np.stack((chassis_x, chassis_y))
    barre_intermediaire = np.stack((barre_intermediaire_x, barre_intermediaire_y))
    effecteur = np.stack((effecteur_x, effecteur_y))

    return chassis, barre_intermediaire, effecteur

def env_p(tvp):
    tvp.flatten()

    p_set_x = np.array([
        cercle_bleu['r']*np.cos(tvp[0]),
        cercle_bleu['r']*np.cos(tvp[0])
    ])
    p_set_y = np.array([
        cercle_bleu['r']*np.sin(tvp[0]),
        cercle_bleu['r']*np.sin(tvp[0])
    ])

    p_set = np.stack((p_set_x, p_set_y))
    return p_set

In [301]:
%%capture

fig = plt.figure(figsize=(18,9))

ax1 = plt.subplot2grid((4, 2), (0, 0), rowspan=5)
ax2 = plt.subplot2grid((4, 2), (0, 1))
ax3 = plt.subplot2grid((4, 2), (1, 1))
ax4 = plt.subplot2grid((4, 2), (2, 1))
ax5 = plt.subplot2grid((4, 2), (3, 1))

ax2.set_ylabel('$d_{0 1 2 3}$ [m]')
ax4.set_ylabel('$p_{x}$ [m]')
ax5.set_ylabel('$p_{y}$ [m]')
ax3.set_ylabel('$\Theta$ [rad]')

# Axis on the right.
for ax in [ax2, ax3, ax4, ax5]:
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    if ax != ax5:
        ax.xaxis.set_ticklabels([])
        
ax5.set_xlabel('time [s]')

mpc_graphics.add_line(var_type='_aux', var_name='d0', axis=ax2, label='d0')
mpc_graphics.add_line(var_type='_aux', var_name='d1', axis=ax2, label='d1')
mpc_graphics.add_line(var_type='_aux', var_name='d2', axis=ax2, label='d2')
mpc_graphics.add_line(var_type='_aux', var_name='d3', axis=ax2, label='d3')
ax2.axhline(0, color='black', linewidth=4)

mpc_graphics.add_line(var_type='_aux', var_name='theta', axis=ax3)
mpc_graphics.add_line(var_type='_aux', var_name='theta_cmd', axis=ax3)
mpc_graphics.add_line(var_type='_aux', var_name='p_x', axis=ax4)
mpc_graphics.add_line(var_type='_aux', var_name='p_x_set', axis=ax4)
mpc_graphics.add_line(var_type='_aux', var_name='p_y', axis=ax5)
mpc_graphics.add_line(var_type='_aux', var_name='p_y_set', axis=ax5)

ax1.axhline(0,color='black')

bar1 = ax1.plot([],[], '-o', linewidth=5, markersize=10, color='black')
bar2 = ax1.plot([],[], '-o', linewidth=5, markersize=10)
bar3 = ax1.plot([],[], '-o', linewidth=5, markersize=10)

point1 = ax1.plot([],[], '-o', linewidth=5, markersize=10, color='red')

if use_obstacle:
    circle = Circle((obstacle['x'], obstacle['y']), obstacle['r'])
    ax1.add_artist(circle)
circle = Circle((cercle_bleu['x'], cercle_bleu['y']), cercle_bleu['r'], edgecolor='blue', fill=False, linestyle='--', linewidth=5)
ax1.add_artist(circle)

size_ax = 3
ax1.set_xlim(-size_ax,size_ax)
ax1.set_ylim(-size_ax,size_ax)
ax1.set_axis_off()

fig.align_ylabels()
fig.tight_layout()

-----------------------
### <center>SIMULATION EN BOUCLE FERMEE</center>
-----------------------
Attention : on fait l'hypothèse qu'on à un retour de létat direct et donc qu'on n'a pas besoin de mettre en place un estimateur.

In [302]:
# Choix des CI

if CI == 'CInulles':
    simulator.x0['x1'] = 0
    simulator.x0['x2'] = 0
    simulator.x0['x3'] = 0
    simulator.x0['x4'] = 0
    simulator.x0['x5'] = 0
    simulator.x0['v1'] = 0
    simulator.x0['v2'] = 0
    simulator.x0['v3'] = 0
    simulator.x0['v4'] = 0
    simulator.x0['a1'] = 0
    simulator.x0['a2'] = 0
    simulator.x0['a3'] = 0
    simulator.x0['a4'] = 0
elif CI == 'CI1':
    simulator.x0['x1'] = -2
    simulator.x0['x2'] = 1
    simulator.x0['x3'] = 0
    simulator.x0['x4'] = 0
    simulator.x0['x5'] = 0
    simulator.x0['v1'] = 0
    simulator.x0['v2'] = 0
    simulator.x0['v3'] = 0
    simulator.x0['v4'] = 0
    simulator.x0['a1'] = 0
    simulator.x0['a2'] = 0
    simulator.x0['a3'] = 0
    simulator.x0['a4'] = 0
elif CI == 'CIcontournement':
    simulator.x0['x1'] = 0
    simulator.x0['x2'] = -1
    simulator.x0['x3'] = -np.pi/2
    simulator.x0['x4'] = 0
    simulator.x0['x5'] = 0
    simulator.x0['v1'] = 0
    simulator.x0['v2'] = 0
    simulator.x0['v3'] = 0
    simulator.x0['v4'] = 0
    simulator.x0['a1'] = 0
    simulator.x0['a2'] = 0
    simulator.x0['a3'] = 0
    simulator.x0['a4'] = 0
elif CI == 'CIcontournement_replie':
    simulator.x0['x1'] = 0
    simulator.x0['x2'] = -1
    simulator.x0['x3'] = -np.pi/2
    simulator.x0['x4'] = np.pi
    simulator.x0['x5'] = np.pi
    simulator.x0['v1'] = 0
    simulator.x0['v2'] = 0
    simulator.x0['v3'] = 0
    simulator.x0['v4'] = 0
    simulator.x0['a1'] = 0
    simulator.x0['a2'] = 0
    simulator.x0['a3'] = 0
    simulator.x0['a4'] = 0
else:
    raise Exception('CI not defined.')

In [303]:
# Declaration de l'etat initial
x0 = simulator.x0.cat.full()
mpc.x0 = x0
estimator.x0 = x0
mpc.set_initial_guess()

In [304]:
%%capture
# Launch simulation

mpc.reset_history()
time_list = []
tic = time.time()  # Start timing
for k in range(n_step_sim):
    # tic = time.time()
    u0 = mpc.make_step(x0)
    # toc = time.time()
    y_next = simulator.make_step(u0)
    x0 = estimator.make_step(y_next)
toc = time.time()  # End timing
# Create a unique key for the current horizon
destination_x = cercle_bleu['r'] * np.cos(consigne1)
destination_y = cercle_bleu['r'] * np.sin(consigne1)
results[f"horizon_{n_horizon}"] = {
    '_x': mpc.data['_x'],  # State data
    '_u': mpc.data['_u'],  # Control input data
    'destination': (destination_x, destination_y)  # Destination finale
}
computation_times[f"horizon_{n_horizon}"] = toc - tic  # Simulation time

# Save the results dictionary for later plotting
np.savez(f'results_data_{n_horizon}.npz', results=results, computation_times=computation_times)

In [305]:
'''
from matplotlib.animation import PillowWriter, ImageMagickWriter

# The function describing the gif:
x_arr = mpc.data['_x']
tvp_arr = mpc.data['_tvp']
def update(t_ind):
    chassis, barre_intermediaire, effecteur = robot(x_arr[t_ind])
    bar1[0].set_data(chassis[0],chassis[1])
    bar2[0].set_data(barre_intermediaire[0],barre_intermediaire[1])
    bar3[0].set_data(effecteur[0],effecteur[1])
    if t_ind==0:
        p_set = env_p(tvp_arr[t_ind])
        point1[0].set_data(p_set[0],p_set[1])
    elif reference:
        p_set = env_p(tvp_arr[t_ind])
        point1[0].set_data(p_set[0],p_set[1])
    mpc_graphics.plot_results(t_ind)
    mpc_graphics.plot_predictions(t_ind)
    mpc_graphics.reset_axes()
    
anim = FuncAnimation(fig, update, frames=n_step_sim, repeat=False)
gif_writer = PillowWriter(fps=10)

filename = f"results/{auteur}__jerk__S{scenario}__{use_objective}__H-{n_horizon}__T-{t_step_ech:.2f}"
filename = filename.replace('.', ',')
filename += '.gif'
anim.save(filename, writer=gif_writer)
print(f"GIF enregistré sous : {filename}")
#'''

'\nfrom matplotlib.animation import PillowWriter, ImageMagickWriter\n\n# The function describing the gif:\nx_arr = mpc.data[\'_x\']\ntvp_arr = mpc.data[\'_tvp\']\ndef update(t_ind):\n    chassis, barre_intermediaire, effecteur = robot(x_arr[t_ind])\n    bar1[0].set_data(chassis[0],chassis[1])\n    bar2[0].set_data(barre_intermediaire[0],barre_intermediaire[1])\n    bar3[0].set_data(effecteur[0],effecteur[1])\n    if t_ind==0:\n        p_set = env_p(tvp_arr[t_ind])\n        point1[0].set_data(p_set[0],p_set[1])\n    elif reference:\n        p_set = env_p(tvp_arr[t_ind])\n        point1[0].set_data(p_set[0],p_set[1])\n    mpc_graphics.plot_results(t_ind)\n    mpc_graphics.plot_predictions(t_ind)\n    mpc_graphics.reset_axes()\n    \nanim = FuncAnimation(fig, update, frames=n_step_sim, repeat=False)\ngif_writer = PillowWriter(fps=10)\n\nfilename = f"results/{auteur}__jerk__S{scenario}__{use_objective}__H-{n_horizon}__T-{t_step_ech:.2f}"\nfilename = filename.replace(\'.\', \',\')\nfilename